#### **Bước 1. Phân tích các phương thức có trong chương trình**
**Bài toán:** Tôi tìm thấy mật mã này trong một cuốn sách cũ. Bạn có thể hiểu được nó nói gì không? Kết nối với `nc jupiter.challenges.picoctf.org 5726`.

**Hint:** 
- Có những công cụ giúp việc này trở nên dễ dàng.
- Có lẽ việc nhìn lại lịch sử sẽ giúp ích


**Phân tích**

Nhìn vào bài toán, ta có thể thấy, đây có thể làm một bài toán mã hóa thay thế.

Bây  giờ, trước hết, ta lấy dữ liệu cần mã hóa:

Nhìn vào flag, ta có thể thấy, đây không phải là mật mã thay thế. Ta có thể sử dụng `Vigenère` để thử giải mã

In [2]:
from pwn import *

# Tạo một conection remote:
port = 5726
r = remote('jupiter.challenges.picoctf.org', port)

get_success = r.recvuntil('Encrypted message:\n').decode()
cipher = r.recvuntil('\n\n\n').decode()
r.close()

[x] Opening connection to jupiter.challenges.picoctf.org on port 5726
[x] Opening connection to jupiter.challenges.picoctf.org on port 5726: Trying 3.131.60.8
[+] Opening connection to jupiter.challenges.picoctf.org on port 5726: Done


C:\Users\Dell\AppData\Local\Temp\ipykernel_13828\2371226947.py:7: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  get_success = r.recvuntil('Encrypted message:\n').decode()


[*] Closed connection to jupiter.challenges.picoctf.org port 5726


C:\Users\Dell\AppData\Local\Temp\ipykernel_13828\2371226947.py:8: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  cipher = r.recvuntil('\n\n\n').decode()


Nhìn vào flag, có thể đoán ra, key chỉ gồm 4 chữ cái, và trong đó sẽ có một chữ `a`, hãy thử hoán vị từ `flag` và tính toán

**Tham khảo**   https://github.com/kevinjycui/picoCTF-2019-writeup/blob/master/Cryptography/la%20cifra%20de/README.md 

In [19]:
flag_enc = "pohzCZK{m311a50_0x_a1rn3x3_h1ah3x6kp60egf}"
flag     = "picoCTF{.................................}"

def vigenere_encrypt(plaintext, key):
    key = key.lower()  # Khóa được chuyển sang chữ thường
    ciphertext = []
    key_index = 0
    
    for char in plaintext:
        if char.isalpha():  
            if char.islower():
                shift = (ord(char) - ord('a') + ord(key[key_index]) - ord('a')) % 26
                ciphertext.append(chr(shift + ord('a')))
            else:
                shift = (ord(char.lower()) - ord('A') + ord(key[key_index]) - ord('a')) % 26
                ciphertext.append(chr(shift + ord('A')))
            
            key_index = (key_index + 1) % len(key)  # Dịch chuyển trong khóa
        else:
            ciphertext.append(char)  # Bỏ qua ký tự không phải chữ cái
    
    return ''.join(ciphertext)

def vigenere_decrypt(ciphertext, key):
    key = key.lower()  # Khóa được chuyển sang chữ thường
    # ciphertext = ciphertext.lower()
    plaintext = []
    key_index = 0
    
    for char in ciphertext:
        if char.isalpha(): 
            if char.islower():
                shift = (ord(char) - ord(key[key_index]) + 26) % 26
                plaintext.append(chr(shift + ord('a')))
            else:
                shift = (ord(char.lower()) - ord(key[key_index]) + 26) % 26
                plaintext.append(chr(shift + ord('A')))
            
            key_index = (key_index + 1) % len(key) 
        else:
            plaintext.append(char)  # Bỏ qua ký tự không phải chữ cái
    
    return ''.join(plaintext)

key = "agfl"
ciphertext = cipher
flag = vigenere_decrypt(flag_enc, key)

print(flag)

picoCTF{b311a50_0r_v1gn3r3_c1ph3r6fe60eaa}
